In [1]:
import os

import discoursegraphs as dg
import discourseinfostat as di

In [2]:
TUEBADZ8_FILE = os.path.expanduser(
    '~/corpora/tuebadz-8.0/tuebadz-8.0-mit-NE+Anaphern+Diskurs.exml.xml')

In [3]:
corpus = dg.read_exportxml(TUEBADZ8_FILE, debug=False)

In [4]:
doc = corpus.next()

In [5]:
debug_corpus = dg.read_exportxml(TUEBADZ8_FILE, debug=True)
debug_doc = debug_corpus.next()
edg = dg.readwrite.exportxml.ExportXMLDocumentGraph(debug_doc)

In [6]:
# %matplotlib inline
# %load_ext gvmagic

In [7]:
# %dotstr dg.print_dot(doc)

In [8]:
# for i, doc in enumerate(corpus):
#     try:
#         dg.print_dot(doc)
# #         print i, "sucess"
#     except:
#         print i, "failed"

In [9]:
# dg.info(doc)

In [ ]:
# import itertools

# onset = 0
# offset = 0

# for token_id, token_str in itertools.islice(doc.get_tokens(), 5):
#     offset = onset + len(token_str)
#     print onset, token_str, offset
#     doc.node[token_id]['{}:{}'.format(doc.ns, 'onset')] = onset
#     doc.node[token_id]['{}:{}'.format(doc.ns, 'offset')] = offset
#     print doc.node[token_id]
#     onset = offset + 1

In [ ]:
# print dg.get_text(doc)

# TODO: difference between `exportxml:coreference` and `exportxml:coreferential`?

The corpus metadata only has `coreferential`.  
I introduced `coreference` myself; this layer will be added  
to all `<relation>` with a `target` attribute, as well as all
`<splitRelation>`s.

```xml
<edge name="relation" parent="word|node">
  <enum-attr name="type">
   <val name="anaphoric" description="Anaphorisches Pronomen"/>
   <val name="cataphoric" description="Kataphorisches Pronomen"/>
   <val name="coreferential" description="Diskurs-altes nicht-Pronomen"/>
  </enum-attr>
  <node-ref name="target"/>
</edge>
```

In [ ]:
# len(dg.get_pointing_chains(doc, layer='exportxml:coreference'))

In [ ]:
# len(dg.get_pointing_chains(doc, layer='exportxml:coreferential'))

In [ ]:
# coreference_set = set(tuple(coref_list)
#                       for coref_list in dg.get_pointing_chains(doc, layer='exportxml:coreference'))

In [ ]:
# coreferential_set = set(tuple(coref_list)
#                       for coref_list in dg.get_pointing_chains(doc, layer='exportxml:coreferential'))

In [ ]:
# not_in_coreference_set = coreferential_set.difference(coreference_set)
# not_in_coreferential_set = coreference_set.difference(coreferential_set)

# for coref_chain in not_in_coreference_set:
#     print ' -> '.join(dg.get_text(doc, token) for token in coref_chain)
#     print

In [ ]:
# for coref_chain in not_in_coreferential_set:
#     print ' -> '.join(dg.get_text(doc, token) for token in coref_chain)
#     print

# FAIL: both sets overlap and the chains in `coreference` often seem to subsume the ones from `coreferential`

This is how relations are specified in Tueba:

```xml
<edge name="relation" parent="word|node">
  <enum-attr name="type">
   <val name="anaphoric" description="Anaphorisches Pronomen"/>
   <val name="cataphoric" description="Kataphorisches Pronomen"/>
   <val name="coreferential" description="Diskurs-altes nicht-Pronomen"/>
  </enum-attr>
  <node-ref name="target"/>
</edge>
```

## TODO: find out where in the code `coreferential` edges are constructed


The parent node of a ``<relation>`` element will have a ``{'relation' : 'coreferential'}`` attribute,  
as well as an ``exportxml:coreferential`` layer.

```python
self.node[parent_node_id].update({'relation': reltype})
self.add_layer(parent_node_id, self.ns+':'+reltype)
```

for each ``<relation>`` with a ``target`` attribute,  
``add_relation()`` adds an edge with both its reltype  
as well as ``exportxml:coreference`` as layers.

```python
# reltype: anaphoric, cataphoric, coreferential
self.add_edge(parent_node_id, target_id,
              layers={self.ns, self.ns+':'+reltype,
                      self.ns+':coreference'},
              label=reltype,
              edge_type=dg.EdgeTypes.pointing_relation)
```

# Conclusion:

## stick with exportxml:coreference
## use exportxml:coreferential/anaphoric/cataphoric only for subclassification / error analysis

# TODO: generic way to extract NPs from corpora with dg?

- NPs have the node label `NX` in Tüba/DZ

In [20]:
%%time

first_docs = list(itertools.islice(corpus, 5))

# previous_mentions, term_count = di.corpus_ngram_frequencies(first_docs)
previous_mentions, term_count = di.corpus_ngram_frequencies(corpus)
#np_sums, np_means = di.term_tfidf(first_docs, previous_mentions, term_count)
np_sums, np_means = di.term_tfidf(corpus, previous_mentions, term_count)


/home/arne/.virtualenvs/notebook/local/lib/python2.7/site-packages/discoursegraphs-0.1.2-py2.7.egg/discoursegraphs/readwrite/exportxml.py:604: UserWarning: <node> element is not a descendant of a <sentence> We'll try to extract the sentence ID from the prefix of the element ID
  "prefix of the element ID".format(element.tag))


KeyboardInterrupt: 